In [ ]:
# long only strategy
V0 = 200000
L = 5
#innitialize the dataframe
data['capital profit'] = np.zeros(len(data))
data['capital profit'][0] = V0 * data['Rate (%)'][0] * 0.01
data['PnL'] = np.zeros(len(data))
data['PnL'][0] = data['capital profit'][0]
data['position'] = np.zeros(len(data))
data['cash'] = np.zeros(len(data))
data['Theta'] = np.zeros(len(data))
data['cash'][0] = V0 + data['capital profit'][0]


sell_entry = 0
buy_entry = 0

for i in range(1,19):
    data['position'][i] = 0
    data['cash'][i] = data['cash'][i-1]
    data['capital profit'][i] = data['cash'][i] * data['Rate (%)'][i] * 0.01
    data['PnL'][i] = data['capital profit'][i]
    data['cash'][i] = data['cash'][i] + data['PnL'][i]
    data['Theta'][i] = data['position'][i]* data['Close'][i]


for i in range(19, len(data)):
    #exit the position

    if buy_entry != 0:
        if data['PnL'][i-5:i].sum()< data['Theta'][i-1] * -0.2 :
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            buy_entry = 0
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
    elif sell_entry != 0:
        if data['PnL'][i-5:i].sum()< abs(data['Theta'][i-1]) * -0.2 :
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            sell_entry = 0
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
 
    if data['Signal'][i] == 1 and buy_entry == 0 and data['position'][i-1] >= 0:
        buy_entry = data['Close'][i]
        data['position'][i] = (L * data['cash'][i-1] * 0.65) / data['Close'][i]
        data['cash'][i] = data['cash'][i-1] * 0.35
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['position'][i] *data['Close'][i] * data['Excess return'][i] + data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])
        
    elif data['Signal'][i] == -1 and sell_entry == 0 and data['position'][i-1] <= 0:
        sell_entry = data['Close'][i]
        data['position'][i] = - (L * data['cash'][i-1] * 0.65) / data['Close'][i]
        data['cash'][i] = data['cash'][i-1] * 0.35
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['position'][i] *data['Close'][i] * data['Excess return'][i] + data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])
    
    elif buy_entry != 0 and data['Close'][i] > data['MA20'][i]:
        data['position'][i] = 0
        data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * buy_entry/L
        #data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        buy_entry = 0
        #pnlcache.append(data['PnL'][i])
        
    elif sell_entry != 0 and data['Close'][i] < data['MA20'][i]:
        data['position'][i] = 0
        data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * sell_entry/L
        #data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        sell_entry = 0
        #pnlcache.append(data['PnL'][i])
        
    else:
        data['position'][i] = data['position'][i-1]
        data['cash'][i] = data['cash'][i-1]
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i] + data['position'][i] *data['Close'][i] * data['Excess return'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])

    if i == len(data)-1:
        if buy_entry != 0:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
        if sell_entry != 0:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
    
    data['Theta'][i] = data['position'][i]* data['Close'][i]

In [ ]:
    """
    this part is not working, changed to the following
    if buy_entry != 0:
        if data['PnL'][buy_date:i+1].sum() < data['Theta'][buy_date] * -0.1 :
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * buy_entry/L
            buy_entry = 0
            buy_date = -1
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
    
    elif sell_entry != 0:
        if data['PnL'][sell_date:i+1].sum() < abs(data['Theta'][sell_date]) * -0.1:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * sell_entry/L
            sell_entry = 0
            sell_date = -1
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
    """

In [ ]:
# Retrieve the data for the SPDR
data = training_set.copy()


# Compute the 20-day moving average
data['MA20'] = data['Close'].rolling(window=20).mean()

# Compute the 20-day standard deviation
data['STD20'] = data['Close'].rolling(window=20).std()

# Compute the upper and lower Bollinger Bands
data['UpperBand'] = data['MA20'] + 2 * data['STD20']
data['LowerBand'] = data['MA20'] - 2 * data['STD20']

# use bollinger bands to determine the signal long at buy signal and short at sell signal
data['Signal'] = 0.0
data['Signal'] = np.where(data['Close'] > data['UpperBand'], -1.0, 0.0)
data['Signal'] = np.where(data['Close'] < data['LowerBand'], 1.0, data['Signal'])

V0 = 200000
L = 5
#innitialize the dataframe
data['capital profit'] = np.zeros(len(data))
data['capital profit'][0] = V0 * data['Rate (%)'][0] * 0.01
data['PnL'] = np.zeros(len(data))
data['PnL'][0] = data['capital profit'][0]
data['position'] = np.zeros(len(data))
data['cash'] = np.zeros(len(data))
data['Theta'] = np.zeros(len(data))
data['cash'][0] = V0 + data['capital profit'][0]
data['V'] = np.zeros(len(data))
data['V'][0] = V0

sell_entry = 0
buy_entry = 0

for i in range(1,19):
    data['position'][i] = 0
    data['cash'][i] = data['cash'][i-1]
    data['capital profit'][i] = data['cash'][i] * data['Rate (%)'][i] * 0.01
    data['PnL'][i] = data['capital profit'][i]
    data['cash'][i] = data['cash'][i] + data['PnL'][i]
    data['Theta'][i] = data['position'][i]* data['Close'][i]
    data['V'][i] = data['V'][i-1] + data['PnL'][i]


for i in range(19, len(data)):
    #exit the position
    if data['Signal'][i] == 1 and buy_entry == 0 and data['position'][i-1] >= 0:
        buy_entry = data['Close'][i]
        buy_date = i
        data['position'][i] = math.floor((L * data['cash'][i-1] * 0.7) / data['Close'][i])
        data['cash'][i] = data['cash'][i-1] - data['position'][i] * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['position'][i] *data['Close'][i] * data['Excess return'][i] + data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])

    elif data['Signal'][i] == -1 and sell_entry == 0 and data['position'][i-1] <= 0:
        sell_entry = data['Close'][i]
        sell_date = i
        data['position'][i] = - math.floor((L * data['cash'][i-1] * 0.7) / data['Close'][i])
        data['cash'][i] = data['cash'][i-1] - abs(data['position'][i]) * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['position'][i] *data['Close'][i] * data['Excess return'][i] + data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])
    
    elif buy_entry != 0 and data['Close'][i] > data['MA20'][i]  + 0.5*data['STD20'][i]:
        data['position'][i] = 0
        data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * buy_entry/L
        #data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        buy_entry = 0
        #pnlcache.append(data['PnL'][i])
        
    elif sell_entry != 0 and data['Close'][i] < data['MA20'][i] - 0.5*data['STD20'][i]:
        data['position'][i] = 0
        data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * sell_entry/L
        #data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        sell_entry = 0
        #pnlcache.append(data['PnL'][i])
    
    elif buy_entry != 0 and data['PnL'][buy_date:i].sum() < data['Theta'][buy_date] * -0.05:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * buy_entry/L
            buy_entry = 0
            buy_date = -1
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
    
    elif sell_entry != 0 and data['PnL'][sell_date:i].sum() < abs(data['Theta'][sell_date]) * -0.05:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * sell_entry/L
            sell_entry = 0
            sell_date = -1
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
            #pnlcache.append(data['PnL'][i])
    else:
        data['position'][i] = data['position'][i-1]
        data['cash'][i] = data['cash'][i-1]
        data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
        data['PnL'][i] = data['capital profit'][i] + data['position'][i] *data['Close'][i] * data['Excess return'][i]
        data['cash'][i] = data['cash'][i] + data['PnL'][i]
        #pnlcache.append(data['PnL'][i])

    if i == len(data)-1:
        if buy_entry != 0:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
        if sell_entry != 0:
            data['position'][i] = 0
            data['cash'][i] = data['cash'][i-1] + abs(data['position'][i-1]) * data['Close'][i]/L
            data['capital profit'][i] = data['Rate (%)'][i] * 0.01 * data['cash'][i]
            data['PnL'][i] = data['capital profit'][i]
            data['cash'][i] = data['cash'][i] + data['PnL'][i]
    
    data['Theta'][i] = data['position'][i]* data['Close'][i]
    data['V'][i] = data['V'][i-1] + data['PnL'][i]

In [ ]:
V0 = 200000
L = 5
#innitialize the dataframe
data2['capital profit'] = np.zeros(len(data2))
data2['PnL'] = np.zeros(len(data2))
data2['capital profit'][0] = data2['Rate (%)'][0] * 0.01 * V0
data2['PnL'][0] = data2['capital profit'][0]
data2['position'] = np.zeros(len(data2))
data2['cash'] = np.zeros(len(data2))
data2['cash'][0] = V0
data2['Theta'] = np.zeros(len(data2))
data2['V'] = np.zeros(len(data2))
data2['cash'][0] = data2['cash'][0] + data2['PnL'][0]
data2['Theta'][0] = data2['position'][0]* data2['Close'][0]
data2['V'][0] = V0 + data2['PnL'][0]
buy_entry = 0
sell_entry = 0

for i in range(1,rolling_window-1):
    data2['position'][i] = 0
    data2['cash'][i] = data2['cash'][i-1]
    data2['capital profit'][i] = data2['Rate (%)'][i] * 0.01 * data2['cash'][i]
    data2['PnL'][i] = data2['capital profit'][i]
    data2['cash'][i] = data2['cash'][i] + data2['PnL'][i]
    data2['Theta'][i] = data2['position'][i]* data2['Close'][i]
    data2['V'][i] = data2['V'][i-1] + data2['PnL'][i]

for i in range(rolling_window-1, len(data2)):
    if data2['signal'][i] == 1 and data2['signal'][i] != data2['signal'][i-1]:
        data2['position'][i] = 0
        data2['cash'][i] = data2['cash'][i-1] + abs(data2['position'][i-1]) * sell_entry/L
        sell_entry = 0
        sell_date = -1
        buy_entry = data2['Close'][i]
        buy_date = i
        data2['position'][i] = math.floor((L * data2['cash'][i] * 0.7) / data2['Close'][i])
        data2['cash'][i] = data2['cash'][i] - data2['position'][i] * data2['Close'][i]/L
        data2['capital profit'][i] = data2['Rate (%)'][i] * 0.01 * data2['cash'][i]
        data2['PnL'][i] = data2['position'][i] *data2['Close'][i] * data2['Excess return'][i] + data2['capital profit'][i]
        data2['cash'][i] = data2['cash'][i] + data2['PnL'][i]
    
    elif data2['signal'][i] == -1 and data2['signal'][i] != data2['signal'][i-1]:
        data2['position'][i] = 0
        data2['cash'][i] = data2['cash'][i-1] + abs(data2['position'][i-1]) * buy_entry/L
        buy_entry = 0
        buy_date = -1
        sell_entry = data2['Close'][i]
        sell_date = i
        data2['position'][i] = -math.floor((L * data2['cash'][i] * 0.7) / data2['Close'][i])
        data2['cash'][i] = data2['cash'][i] - abs(data2['position'][i]) * data2['Close'][i]/L
        data2['capital profit'][i] = data2['Rate (%)'][i] * 0.01 * data2['cash'][i]
        data2['PnL'][i] = data2['position'][i] *data2['Close'][i] * data2['Excess return'][i] + data2['capital profit'][i]
        data2['cash'][i] = data2['cash'][i] + data2['PnL'][i]

    #if the signal is the same as the previous day, then the position and cash remain the same
    elif data2['signal'][i] == data2['signal'][i-1] == 1:
        data2['position'][i] = data2['position'][i-1]
        data2['cash'][i] = data2['cash'][i-1]
        data2['capital profit'][i] = data2['Rate (%)'][i] * 0.01 * data2['cash'][i]
        data2['PnL'][i] = data2['position'][i] *data2['Close'][i] * data2['Excess return'][i] + data2['capital profit'][i]
        data2['cash'][i] = data2['cash'][i] + data2['PnL'][i]
    #if the signal is the same as the previous day, then the position and cash remain the same
    elif data2['signal'][i] == data2['signal'][i-1] == -1:
        data2['position'][i] = data2['position'][i-1]
        data2['cash'][i] = data2['cash'][i-1]
        data2['capital profit'][i] = data2['Rate (%)'][i] * 0.01 * data2['cash'][i]
        data2['PnL'][i] = data2['position'][i] *data2['Close'][i] * data2['Excess return'][i] + data2['capital profit'][i]
        data2['cash'][i] = data2['cash'][i] + data2['PnL'][i]
    
    data2['Theta'][i] = data2['position'][i]* data2['Close'][i]
    data2['V'][i] = data2['V'][i-1] + data2['PnL'][i]